In [1]:
import json
import boto3
import sagemaker as sage
from sagemaker.pytorch import PyTorch
from sagemaker.estimator import Estimator

boto_session = boto3.Session()
sess = sage.Session(boto_session=boto_session)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\ayakovyna\AppData\Local\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\ayakovyna\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
iam = boto3.client('iam', region_name='eu-central-1')
role_name = 'flexki-nano-demo-test-role-hamza'
policy = {
    'Statement': [{
        'Action': 'sts:AssumeRole',
        'Effect': 'Allow',
        'Principal': {'Service': 'sagemaker.amazonaws.com'}},
    ],  
    'Version': '2012-10-17'
}

roles = iam.list_roles()
role_arn = None
for role in roles['Roles']:
    if role['RoleName'] == role_name:
        role_arn = role['Arn']
        
if role_arn == None:
    new_role = iam.create_role(
        AssumeRolePolicyDocument=json.dumps(policy),
        Path='/',
        RoleName=role_name,
    )
    role_arn = new_role['Role']['Arn']

    iam.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
    )

    iam.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
    )

In [27]:
estimator = PyTorch(
    entry_point='render-entry.py',
    source_dir='./',
    role=role_arn,
    framework_version="1.12.1",
    py_version="py38",
    instance_count=1,
    # instance_type='ml.p3.16xlarge', # 64 GB, 4 GPUs
    # instance_type='ml.g4dn.xlarge', # 16 GB, fast launch
    instance_type='ml.g5.2xlarge' # 24 GB
    keep_alive_period_in_seconds=120,
    # hyperparameters={
    #     "s": "/opt/ml/input/data/train/mipnerf/kitchen",
    #     "m": "output/mipnerf/kitchen",
    #     "config_file": "config/gaussian_dataset/train.json",
    # },
    base_job_name='3dgaussians-run',
    # distribution={ "smdistributed": { "dataparallel": { "enabled": True } } },
)
# estimator.fit({'train': 's3://flexki-andrii/3dgaussians/'})
estimator.fit({
    'outp': 's3://sagemaker-eu-central-1-182918105663/3dgaussians-run-2024-03-29-15-59-38-622/output/output.tar.gz',
    'train': 's3://flexki-andrii/3dgaussians/'
})

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\ayakovyna\AppData\Local\sagemaker\sagemaker\config.yaml


INFO:botocore.tokens:Loading cached SSO token for flex-ki


Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: 3dgaussians-run-2024-04-01-07-20-56-307


2024-04-01 07:21:54 Starting - Starting the training job...
2024-04-01 07:22:08 Starting - Preparing the instances for training...
2024-04-01 07:22:57 Downloading - Downloading input data.....
2024-04-01 07:23:57 Downloading - Downloading the training image............
2024-04-01 07:26:18 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-04-01 07:26:27,312 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-04-01 07:26:27,339 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-04-01 07:26:27,350 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-04-01 07:26:27,354 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-04-01 07:26:28,733 sagemaker-training-toolkit INFO     Installing dependencies from

EndpointConnectionError: Could not connect to the endpoint URL: "https://api.sagemaker.eu-central-1.amazonaws.com/"